TO DO:
- Figure out what years to restrict data
- Look into mixed data type columns in birding data
- Look into “sensitive species” list
- Sift through tables of interest for FIA
- Figure out how to read in specific tabs in the xlsx files for FIA
- Clean tables to drop extraneous columns
- EDA on data once cleaned
- Consider geospatial uses of data
- Separate observation date into year columm (and maybe month) to group by 
- Value counts on bird species to get top 10 or so to track over years
- Merge birds table with USFWS table to get specific region & download doc off data.gov website



COMPLETED:

<b>X</b> Write a read me

General notes:
- 1st choice: group by wildlife region; else group by county
- try to webscrape polygons off of USFWS website

In [35]:
import pandas as pd
import numpy as np

In [ ]:
# trees TN 2016

# create a dicitonary (with sheet_name =['<sheet1>', '<sheet2>', '<sheet3>'])
# use .get('<sheetname>') to parse into separate dfs

dict_trees2016 = pd.read_excel('../data/TN_2016.xlsx',
             sheet_name = ['Tab 2'])

In [31]:
# reading in USFWS codes
usfws_codes = pd.read_csv('../data/eBird_2010_to_2014_TN/USFWSCodes.txt', sep='\t')

# cleaning usfws codes df
usfws_codes = usfws_codes.reset_index()
usfws_codes['usfws_name'] = np.where(usfws_codes['USFWS NAME'].isnull(), usfws_codes['USFWS CODE'], usfws_codes['USFWS NAME'])
usfws_codes = usfws_codes[['index','usfws_name']]
usfws_codes.columns = ['usfws_code', 'usfws_name']

In [2]:
# 2010 to 2014
birds_2010_2014 = pd.read_csv('../data/eBird_2010_to_2014_TN/eBird_2010_to_2014_TN.txt', sep='\t')

C:\Users\danil\AppData\Local\Temp\ipykernel_30600\1194818085.py:2: DtypeWarning: Columns (11,12,13,23) have mixed types. Specify dtype option on import or set low_memory=False.
  birds_2010_2014 = pd.read_csv('../data/eBird_2010_to_2014_TN/eBird_2010_to_2014_TN.txt', sep='\t')


In [4]:
short_birds = birds_2010_2014[['GLOBAL UNIQUE IDENTIFIER', 'OBSERVATION DATE', 'TAXONOMIC ORDER', 'CATEGORY', 'COMMON NAME', 'SCIENTIFIC NAME', 'AGE/SEX', 'OBSERVATION COUNT', 'COUNTRY', 'STATE', 'COUNTY', 'IBA CODE', 'BCR CODE', 'USFWS CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVER ID', 'TRIP COMMENTS']]
short_birds.columns = ['global_unique_identifier', 'observation_date', 'tax_order', 'category', 'common_name', 'scientific_name', 'age_sex', 'observation_count', 'country', 'state', 'county', 'iba_code', 'bcr_code', 'usfws_code', 'lat', 'long', 'observer_id', 'trip_comments']
short_birds.head()

,global_unique_identifier,observation_date,tax_order,category,common_name,scientific_name,age_sex,observation_count,country,state,county,iba_code,bcr_code,usfws_code,lat,long,observer_id,trip_comments
0,URN:CornellLabOfOrnithology:EBIRD:OBS81849899,2010-01-19,21054,species,American Crow,Corvus brachyrhynchos,NaN,1,United States,Tennessee,Anderson,NaN,28,NaN,35.958338,-84.246758,obsr18758,beautiful warm sunny day
1,URN:CornellLabOfOrnithology:EBIRD:OBS82004086,2010-01-22,21054,species,American Crow,Corvus brachyrhynchos,NaN,5,United States,Tennessee,Anderson,NaN,28,NaN,36.217869,-84.086437,obsr18758,"Chilly, overcast, mist. After not seeing a wi..."
2,URN:CornellLabOfOrnithology:EBIRD:OBS80663800,2010-01-02,21054,species,American Crow,Corvus brachyrhynchos,NaN,10,United States,Tennessee,Anderson,NaN,28,NaN,35.996341,-84.163513,obsr54372,All within Haw Ridge Park and within Knoxville...
3,URN:CornellLabOfOrnithology:EBIRD:OBS82003744,2010-01-22,21054,species,American Crow,Corvus brachyrhynchos,NaN,3,United States,Tennessee,Anderson,NaN,28,NaN,36.116386,-84.110001,obsr18758,NaN
4,URN:CornellLabOfOrnithology:EBIRD:OBS287169932,2010-01-15,21054,species,American Crow,Corvus brachyrhynchos,NaN,5,United States,Tennessee,Anderson,NaN,28,NaN,36.007052,-84.262025,obsr276568,NaN


In [5]:
## here:
## make separate year column for observation date

short_birds['observation_year'] = [x[:4] for x in short_birds.observation_date]

C:\Users\danil\AppData\Local\Temp\ipykernel_30600\3119811748.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_birds['observation_year'] = [x[:4] for x in short_birds.observation_date]


In [46]:
# creating subset of birds sighted within usfws polygon:
birds_usfws = short_birds.loc[~short_birds.usfws_code.isna()]

# joining birds_usfws with usfws codes
pd.merge(birds_usfws, usfws_codes, on='usfws_code', how='left')

,global_unique_identifier,observation_date,tax_order,category,common_name,scientific_name,age_sex,observation_count,country,state,county,iba_code,bcr_code,usfws_code,lat,long,observer_id,trip_comments,observation_year,usfws_name
0,URN:CornellLabOfOrnithology:EBIRD:OBS586824762,2010-01-05,32449,species,Dark-eyed Junco,Junco hyemalis,NaN,3,United States,Tennessee,Benton,US-TN_2874,24,USFWS_723,35.917859,-87.947961,obsr345239,NaN,2010,TENNESSEE NATIONAL WILDLIFE REFUGE
1,URN:CornellLabOfOrnithology:EBIRD:OBS586824760,2010-01-05,27739,species,Hermit Thrush,Catharus guttatus,NaN,1,United States,Tennessee,Benton,US-TN_2874,24,USFWS_723,35.917859,-87.947961,obsr345239,NaN,2010,TENNESSEE NATIONAL WILDLIFE REFUGE
2,URN:CornellLabOfOrnithology:EBIRD:OBS586824761,2010-01-05,10887,species,Red-bellied Woodpecker,Melanerpes carolinus,NaN,1,United States,Tennessee,Benton,US-TN_2874,24,USFWS_723,35.917859,-87.947961,obsr345239,NaN,2010,TENNESSEE NATIONAL WILDLIFE REFUGE
3,URN:CornellLabOfOrnithology:EBIRD:OBS535722691,2010-01-22,21054,species,American Crow,Corvus brachyrhynchos,NaN,6,United States,Tennessee,Haywood,US-TN_2833,27,USFWS_497,35.511812,-89.262207,obsr60335,NaN,2010,HATCHIE NATIONAL WILDLIFE REFUGE
4,URN:CornellLabOfOrnithology:EBIRD:OBS222344390,2010-01-19,9939,species,Belted Kingfisher,Megaceryle alcyon,NaN,X,United States,Tennessee,Haywood,US-TN_2833,27,USFWS_497,35.511812,-89.262207,obsr242764,Oneil Lake area only,2010,HATCHIE NATIONAL WILDLIFE REFUGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41465,URN:CornellLabOfOrnithology:EBIRD:OBS286829763,2014-12-24,468,species,Gadwall,Mareca strepera,NaN,40,United States,Tennessee,Stewart,US-TN_2840,24,USFWS_302,36.471654,-87.733346,obsr290393,"<br />Submitted from BirdLog NA for iOS, versi...",2014,CROSS CREEKS NATIONAL WILDLIFE REFUGE
41466,URN:CornellLabOfOrnithology:EBIRD:OBS286829764,2014-12-24,505,species,Mallard,Anas platyrhynchos,NaN,10,United States,Tennessee,Stewart,US-TN_2840,24,USFWS_302,36.471654,-87.733346,obsr290393,"<br />Submitted from BirdLog NA for iOS, versi...",2014,CROSS CREEKS NATIONAL WILDLIFE REFUGE
41467,URN:CornellLabOfOrnithology:EBIRD:OBS286829767,2014-12-24,1813,species,Pied-billed Grebe,Podilymbus podiceps,NaN,4,United States,Tennessee,Stewart,US-TN_2840,24,USFWS_302,36.471654,-87.733346,obsr290393,"<br />Submitted from BirdLog NA for iOS, versi...",2014,CROSS CREEKS NATIONAL WILDLIFE REFUGE
41468,URN:CornellLabOfOrnithology:EBIRD:OBS286829765,2014-12-24,617,species,Redhead,Aythya americana,NaN,1,United States,Tennessee,Stewart,US-TN_2840,24,USFWS_302,36.471654,-87.733346,obsr290393,"<br />Submitted from BirdLog NA for iOS, versi...",2014,CROSS CREEKS NATIONAL WILDLIFE REFUGE


## EDA

In [6]:
short_birds.common_name.value_counts().head(10)

Northern Cardinal         48842
American Crow             41626
Carolina Chickadee        40837
Blue Jay                  39305
Carolina Wren             39079
Mourning Dove             36616
Tufted Titmouse           36309
American Robin            35295
Northern Mockingbird      31399
Red-bellied Woodpecker    30536
Name: common_name, dtype: int64

In [7]:
short_birds.groupby(['observation_year', 'common_name']).count()[['observation_count']]

observation_count
observation_year common_name                                                     
2010             Acadian Flycatcher                                           255
                 Accipiter sp.                                                  3
                 Alder Flycatcher                                              18
                 Alder/Willow Flycatcher (Traill's Flycatcher)                  5
                 American Avocet                                                6
...                                                                           ...
2014             tern sp.                                                       2
                 thrush sp.                                                    22
                 vireo sp.                                                      3
                 waterfowl sp.                                                  1
                 woodpecker sp.                                                13

[1799 rows x 1 columns]

In [8]:
# TN 2016 - figure out how to read in specific tabs:

# create a dicitonary (with sheet_name =['<sheet1>', '<sheet2>', '<sheet3>'])
# use .get('<sheetname>') to parse into separate dfs

dict_trees2016 = pd.read_excel('../data/TN_2016.xlsx',
             sheet_name = ['Tab 2'])

In [9]:
# cleaning table 2 - land by ownership class
land_by_ownership_class = dict_trees2016.get('Tab 2')
land_by_ownership_class.columns = ['class_type', 'all_forest_land', 'total_unreserved', 'timberland', 'unproductive', 'total_reserved', 'productive', 'unproductive_reserved']
land_by_ownership_class = land_by_ownership_class.head(28)
land_by_ownership_class = land_by_ownership_class.tail(25)
land_by_ownership_class = land_by_ownership_class.set_index('class_type')

In [10]:
# in thousand acres
land_by_ownership_class

,all_forest_land,total_unreserved,timberland,unproductive,total_reserved,productive,unproductive_reserved
class_type,,,,,,,
U.S. Forest Service,NaN,NaN,NaN,NaN,NaN,NaN,NaN
National forest,654.7,589.8,585.8,4,64.9,62.3,2.6
Other Forest Service,58.1,58.1,58.1,0,0,0,0
Total,712.8,647.9,643.9,4,64.9,62.3,2.6
Other Federal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
National Park Service,335.4,0,0,0,335.4,335.4,0
U.S. Fish and Wildlife Service,30.8,0,0,0,30.8,30.8,0
Dept. of Defense/Dept. of Energy,164.3,162.8,162.8,0,1.5,1.5,0
Other Federal,141,135,135,0,6.1,6.1,0


array(['USFWS_723', 'USFWS_497', 'USFWS_765', 'USFWS_302', 'USFWS_127',
       'USFWS_429', 'USFWS_374', 'USFWS_276', 'USFWS_162'], dtype=object)